In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!unzip /content/drive/MyDrive/Bubble_new.zip

Streaming output truncated to the last 5000 lines.
  inflating: Train/Negative/0.3_3-0063.jpg  
  inflating: __MACOSX/Train/Negative/._0.3_3-0063.jpg  
  inflating: Train/Negative/250_2.2-0021.jpg  
  inflating: __MACOSX/Train/Negative/._250_2.2-0021.jpg  
  inflating: Train/Negative/0.3_6-0069.jpg  
  inflating: __MACOSX/Train/Negative/._0.3_6-0069.jpg  
  inflating: Train/Negative/0.2_1-0128.jpg  
  inflating: __MACOSX/Train/Negative/._0.2_1-0128.jpg  
  inflating: Train/Negative/0.3_1-0540.jpg  
  inflating: __MACOSX/Train/Negative/._0.3_1-0540.jpg  
  inflating: Train/Negative/0.2_3-0184.jpg  
  inflating: __MACOSX/Train/Negative/._0.2_3-0184.jpg  
  inflating: Train/Negative/Control_2.2-0074.jpg  
  inflating: __MACOSX/Train/Negative/._Control_2.2-0074.jpg  
  inflating: Train/Negative/0.1_4-0110.jpg  
  inflating: __MACOSX/Train/Negative/._0.1_4-0110.jpg  
  inflating: Train/Negative/IMG_20220623_015535747.jpg  
  inflating: Train/Negative/0.2_2-0190.jpg  
  inflating: __MACOSX/T

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
image_height, image_width = 299, 299
batch_size = 600



train_ds = tf.keras.utils.image_dataset_from_directory(
    "Train",
    image_size = (image_height, image_width),
    batch_size = batch_size
     )
#val_ds = tf.keras.utils.image_dataset_from_directory(
 #   "Validation",  
  #  image_size = (image_height, image_width),
   # batch_size = batch_size
    # )
test_ds = tf.keras.utils.image_dataset_from_directory(
    "Test",
    image_size = (image_height, image_width),
    batch_size = batch_size
     )

Found 17886 files belonging to 2 classes.
Found 1800 files belonging to 2 classes.


In [4]:
class_names = train_ds.class_names
print(class_names)



['Negative', 'Positive']


In [5]:
model = Sequential()
pretrained_model = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape= (299, 299, 3),
    pooling='max',
    classes=2,
    classifier_activation="softmax",
)
for layer in pretrained_model.layers:
        layer.trainable=False
model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu', kernel_regularizer='l2'))
model.add(Dense(2, activation = 'softmax'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
 module_wrapper (ModuleWrapp  (None, 2048)             0         
 er)                                                             
                                                                 
 module_wrapper_1 (ModuleWra  (None, 256)              524544    
 pper)                                                           
                                                                 
 module_wrapper_2 (ModuleWra  (None, 128)              32896     
 pper)                                                           
                                                                 
 module_wrapper_3 (ModuleWra  (None, 64)               8256      
 pper)                                                  

In [7]:
optimizer = Adam(learning_rate=0.005)
# model.compile(loss=
# tf.keras.losses.SparseCategoricalCrossentropy(
#     from_logits=False,
#     ignore_class=None,
#     reduction=losses_utils.ReductionV2.AUTO,
#     name='sparse_categorical_crossentropy'
# ),
#     optimizer=optimizer,
#     metrics= ['MeanSquaredError','acc'])

model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


In [ ]:

history = model.fit(train_ds, validation_data= test_ds, epochs=100)

Epoch 1/100
30/30 [==============================] - 467s 12s/step - loss: 43.5500 - sparse_categorical_accuracy: 0.5026 - val_loss: 7.5424 - val_sparse_categorical_accuracy: 0.8628
Epoch 2/100
30/30 [==============================] - 433s 11s/step - loss: 11.6931 - sparse_categorical_accuracy: 0.5327 - val_loss: 1.5339 - val_sparse_categorical_accuracy: 0.8633
Epoch 3/100
30/30 [==============================] - 415s 11s/step - loss: 9.1942 - sparse_categorical_accuracy: 0.5400 - val_loss: 6.7715 - val_sparse_categorical_accuracy: 0.8628
Epoch 4/100
24/30 [=======================>......] - ETA: 1:07 - loss: 4.5920 - sparse_categorical_accuracy: 0.5442

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training loss')
plt.legend()
plt.show()

In [ ]:
acc1 = model.evaluate['acc']
#val_acc = history.history['val_acc']
epochs = range(1, len(acc1) + 1)
plt.plot(epochs, acc, label='Test acc')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", 'wb') as f:
  f.write(tflite_model)